In [1]:
import os
import json

import pandas as pd
from langchain.output_parsers import PydanticOutputParser
from langchain_mistralai.chat_models import ChatMistralAI

from dotenv import load_dotenv
from baker.parser import RecipeParser, MESSAGE, CHAT_PROMPT_TEMPLATE
from baker.schemas import Recipe

/Users/vianneymixtur/CustomDesktop/exploration/baker/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [2]:
df = pd.read_json("data/recipes_v1_with_token_counts.json")
df.head()

,title,date,tags,introduction,ingredients,direction,output,ingredients_count,direction_count
0,Creamy Mashed Potatoes,2021-03-12,"[potato, side, cheesefare]",![Creamy Mashed Potatoes](/pix/creamy-mashed-p...,The quantities here are for about four adult p...,1. Peel and cut the potatoes into medium sized...,"{'title': 'Creamy Mashed Potatoes', 'ingredien...",93,184
1,Red Sauce (Ragu all'Italiana),2021-03-16,"[italian, sauce]",My great-grandma's red sauce. All purpose: goo...,- 1/3 lb salt pork - 2 lb chuck roast - 3 Clov...,"1. Mince the garlic, peel the carrot, peel and...","{'title': 'Red Sauce (Ragu all'Italiana)', 'in...",61,117
2,Turmeric Flatbread,2022-08-09,"[bread, turkish]",A great companion to Turkish Red Lentil Soup. ...,- 1 cup Wheat Flour (white or whole) - 1/2 Tbs...,"1. Combine flour, turmeric, salt and baking po...","{'title': 'Turmeric Flatbread', 'ingredients':...",30,90
3,Zurich-Style Meat Saute,2023-01-06,"[beef, cream, swiss, quick]","Originally called ""Züri Gschnätzlets"" (Zurich ...","- 600g\tBeef, finely sliced - 250g\tMushrooms,...",1. Fry the meat in butter at high heat until i...,"{'title': 'Zurich-Style Meat Saute', 'ingredie...",37,126
4,Tuscan Style Pork Roast,2021-03-10,"[italian, pork, roast]",![tuscan-style-pork-roast](/pix/tuscan-style-p...,- 1 pork Roast - 2-3 Tbsp fresh rosemary - 8 c...,1.\tPreheat oven to 275°F (135°C) 2.\tButterfl...,"{'title': 'Tuscan Style Pork Roast', 'ingredie...",38,473


In [3]:
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

In [4]:
llm = ChatMistralAI(api_key=MISTRAL_API_KEY, model_name="open-mixtral-8x7b")
parser = PydanticOutputParser(pydantic_object=Recipe)

In [5]:
recipe_parser = RecipeParser(llm=llm, parser=parser)

In [6]:
df["ingredients"][0]

'The quantities here are for about four adult portions. If you are planning on eating this as a side dish, it might be more like 6-8 portions. * 1kg potatoes * 200ml milk* * 200ml mayonnaise* * ~100g cheese * Garlic powder * 12-16 strips of bacon * Butter * 3-4 green onions * Black pepper * Salt  *You can play with the proportions depending on how creamy or dry you want the mashed potatoes to be.'

In [8]:
df["title"].to_list()

['Creamy Mashed Potatoes',
 "Red Sauce (Ragu all'Italiana)",
 'Turmeric Flatbread',
 'Zurich-Style Meat Saute',
 'Tuscan Style Pork Roast',
 'Smoked Salmon Pasta Primavera',
 'Spicy Mayo',
 'Risengrød',
 'Pilaf',
 'Italian Bread',
 'Pancake',
 'Tanzanian Tea w/ Milk',
 'Cheesy Potato Bake',
 'Spaghetti aglio e olio',
 'Quarkbällchen – Fried curd balls',
 'Mapo Tofu (麻婆豆腐)',
 'Chili Con Carne',
 'Liver and Onions',
 'Korv Stroganoff',
 'Naan Bread',
 'Steak Tartare',
 'Hangover Eggs',
 'Baked Salmon',
 'Oatmeal Cookies',
 'Spiced Apple Pancakes',
 'Beef Jerky',
 'Onion Raitha (Indian Tzatziki)',
 'Cheese',
 'Ravioli',
 'Italian Mulled Wine',
 'Refried Beans',
 'Fall Vegetable and Chickpea Curry',
 'Winter Risotto',
 'Parmesan potatoes',
 'Swedish Pancakes',
 'Chicken Parmesan',
 'Red Bean Buns (豆沙包)',
 'Omelet',
 'Aussie Snags (sausage sizzle)',
 'Bitoque - Portuguese Steak with Beer Sauce',
 'Lasagna',
 'Irish Potato Casserole',
 'Greek Yogurt',
 'Tabouleh',
 'Dried tomato & plum bread

In [9]:
df[df["title"] == 'French Crêpes']

,title,date,tags,introduction,ingredients,direction,output,ingredients_count,direction_count
83,French Crêpes,2021-03-14,"[french, dessert, breakfast, cheesefare]","Like pancakes, but very thin. 15 crêpes.",- 300g white flour - 3 eggs - 60cl milk - 20cl...,"1. Mix flour, salt, eggs and melted butter or ...","{'title': 'French Crêpes', 'ingredients': '- 3...",23,138


In [11]:
df["title"][83]

'French Crêpes'

In [12]:
output = await recipe_parser.run(ingredients=df["ingredients"][83], steps=df["direction"][83])
output

Recipe(name='Crêpes', serving_size=4, ingredients=[Ingredient(id=1, name='white flour', quantity=300.0, unit='g'), Ingredient(id=2, name='eggs', quantity=3.0, unit='unit'), Ingredient(id=3, name='milk', quantity=60.0, unit='cl'), Ingredient(id=4, name='beer', quantity=20.0, unit='cl'), Ingredient(id=5, name='butter', quantity=30.0, unit='g')], steps=[Step(number=1, description='Mix flour, eggs, and melted butter in a bowl.', preparation_time=None, cooking_time=None, used_ingredients=[1, 2, 5]), Step(number=2, description='Slowly add milk and beer until the dough becomes fluid enough.', preparation_time=5, cooking_time=None, used_ingredients=[3, 4]), Step(number=3, description='Let the dough rest for one hour.', preparation_time=60, cooking_time=None, used_ingredients=[]), Step(number=4, description='Cook the crêpe in a flat pan, one ladle at a time.', preparation_time=10, cooking_time=None, used_ingredients=[])], total_preparation_time=None, total_cooking_time=None, comments=['Oil the 

In [17]:
french_crepe = output.model_dump(mode="json")
french_crepe

{'name': 'Crêpes',
 'serving_size': 4,
 'ingredients': [{'id': 1,
   'name': 'white flour',
   'quantity': 300.0,
   'unit': 'g'},
  {'id': 2, 'name': 'eggs', 'quantity': 3.0, 'unit': 'unit'},
  {'id': 3, 'name': 'milk', 'quantity': 60.0, 'unit': 'cl'},
  {'id': 4, 'name': 'beer', 'quantity': 20.0, 'unit': 'cl'},
  {'id': 5, 'name': 'butter', 'quantity': 30.0, 'unit': 'g'}],
 'steps': [{'number': 1,
   'description': 'Mix flour, eggs, and melted butter in a bowl.',
   'preparation_time': None,
   'cooking_time': None,
   'used_ingredients': [1, 2, 5]},
  {'number': 2,
   'description': 'Slowly add milk and beer until the dough becomes fluid enough.',
   'preparation_time': 5,
   'cooking_time': None,
   'used_ingredients': [3, 4]},
  {'number': 3,
   'description': 'Let the dough rest for one hour.',
   'preparation_time': 60,
   'cooking_time': None,
   'used_ingredients': []},
  {'number': 4,
   'description': 'Cook the crêpe in a flat pan, one ladle at a time.',
   'preparation_time

In [22]:
with open("french_crepe.json", "w") as f:
    json.dump(french_crepe, f, ensure_ascii=False, indent=4)

In [9]:
output.dict()

{'name': 'Loaded Mashed Potatoes',
 'serving_size': 4,
 'ingredients': [{'id': 1,
   'name': 'potatoes',
   'quantity': 1000.0,
   'unit': 'grams'},
  {'id': 2, 'name': 'milk', 'quantity': 200.0, 'unit': 'milliliters'},
  {'id': 3, 'name': 'mayonnaise', 'quantity': 200.0, 'unit': 'milliliters'},
  {'id': 4, 'name': 'cheese', 'quantity': 100.0, 'unit': 'grams'},
  {'id': 5, 'name': 'garlic powder', 'quantity': None, 'unit': None},
  {'id': 6, 'name': 'bacon', 'quantity': 12.0, 'unit': 'strips'},
  {'id': 7, 'name': 'butter', 'quantity': None, 'unit': None},
  {'id': 8, 'name': 'green onions', 'quantity': None, 'unit': None},
  {'id': 9, 'name': 'black pepper', 'quantity': None, 'unit': None},
  {'id': 10, 'name': 'salt', 'quantity': None, 'unit': None}],
 'steps': [{'number': 1,
   'description': 'Peel and cut the potatoes into medium-sized pieces.',
   'preparation_time': None,
   'cooking_time': None,
   'used_ingredients': [1]},
  {'number': 2,
   'description': 'Put the potatoes in 

In [13]:
first_10_recipes = df.head(10)
outputs = [output]

In [14]:
len(first_10_recipes)

10

In [15]:
for i in range(1, len(first_10_recipes)):
    try:
        output = await recipe_parser.run(ingredients=first_10_recipes["ingredients"][i], steps=first_10_recipes["direction"][i])
        outputs.append(output)
    except Exception as e:
        print(f"Error: {e}")
    

Invalid json output: {
"name": "Pork Roast with Rosemary and Garlic",
"serving\_size": 6,
"ingredients": [
{"id": 1, "name": "pork roast", "quantity": 1, "unit": "unit"},
{"id": 2, "name": "fresh rosemary", "quantity": 2, "unit": "tablespoons"},
{"id": 3, "name": "garlic", "quantity": 8, "unit": "cloves"},
{"id": 4, "name": "olive oil", "quantity": 0.33, "unit": "cup"},
{"id": 5, "name": "lemon", "quantity": 1, "unit": "unit"},
{"id": 6, "name": "pancetta or bacon", "quantity": 4, "unit": "ounces"},
{"id": 7, "name": "parsley", "quantity": 1, "unit": "teaspoon"}
],
"steps": [
{"number": 1, "description": "Preheat the oven to 275°F (135°C)"},
{"number": 2, "description": "Butterfly the pork roast by cutting into three parts and pounding to an even levelness"},
{"number": 3, "description": "Mince the garlic and rosemary and combine with olive oil and lemon zest in a pan. Simmer for 2-3 minutes and strain, reserving the oil."},
{"number": 4, "description": "Process the pancetta/bacon and 

In [16]:
outputs

[Recipe(name='Loaded Mashed Potatoes', serving_size=4, ingredients=[Ingredient(id=1, name='potatoes', quantity=1000.0, unit='grams'), Ingredient(id=2, name='milk', quantity=200.0, unit='milliliters'), Ingredient(id=3, name='mayonnaise', quantity=200.0, unit='milliliters'), Ingredient(id=4, name='cheese', quantity=100.0, unit='grams'), Ingredient(id=5, name='garlic powder', quantity=None, unit=None), Ingredient(id=6, name='bacon', quantity=12.0, unit='strips'), Ingredient(id=7, name='butter', quantity=None, unit=None), Ingredient(id=8, name='green onions', quantity=None, unit=None), Ingredient(id=9, name='black pepper', quantity=None, unit=None), Ingredient(id=10, name='salt', quantity=None, unit=None)], steps=[Step(number=1, description='Peel and cut the potatoes into medium-sized pieces.', preparation_time=None, cooking_time=None, used_ingredients=[1]), Step(number=2, description='Put the potatoes in a pot with some water so that it covers the potatoes and boil them for about 20-30 mi

In [23]:
outputs[4]

'{\n"name": "Pork Roast with Rosemary and Garlic",\n"serving\\_size": 6,\n"ingredients": [\n{"id": 1, "name": "pork roast", "quantity": 1, "unit": "unit"},\n{"id": 2, "name": "fresh rosemary", "quantity": 2, "unit": "tablespoons"},\n{"id": 3, "name": "garlic", "quantity": 8, "unit": "cloves"},\n{"id": 4, "name": "olive oil", "quantity": 0.33, "unit": "cup"},\n{"id": 5, "name": "lemon", "quantity": 1, "unit": "unit"},\n{"id": 6, "name": "pancetta or bacon", "quantity": 4, "unit": "ounces"},\n{"id": 7, "name": "parsley", "quantity": 1, "unit": "teaspoon"}\n],\n"steps": [\n{"number": 1, "description": "Preheat the oven to 275°F (135°C)"},\n{"number": 2, "description": "Butterfly the pork roast by cutting into three parts and pounding to an even levelness"},\n{"number": 3, "description": "Mince the garlic and rosemary and combine with olive oil and lemon zest in a pan. Simmer for 2-3 minutes and strain, reserving the oil."},\n{"number": 4, "description": "Process the pancetta/bacon and gar

In [27]:
test = outputs[4].replace("\n", "")
test = test.replace("\\", "")
test

'{"name": "Pork Roast with Rosemary and Garlic","serving_size": 6,"ingredients": [{"id": 1, "name": "pork roast", "quantity": 1, "unit": "unit"},{"id": 2, "name": "fresh rosemary", "quantity": 2, "unit": "tablespoons"},{"id": 3, "name": "garlic", "quantity": 8, "unit": "cloves"},{"id": 4, "name": "olive oil", "quantity": 0.33, "unit": "cup"},{"id": 5, "name": "lemon", "quantity": 1, "unit": "unit"},{"id": 6, "name": "pancetta or bacon", "quantity": 4, "unit": "ounces"},{"id": 7, "name": "parsley", "quantity": 1, "unit": "teaspoon"}],"steps": [{"number": 1, "description": "Preheat the oven to 275°F (135°C)"},{"number": 2, "description": "Butterfly the pork roast by cutting into three parts and pounding to an even levelness"},{"number": 3, "description": "Mince the garlic and rosemary and combine with olive oil and lemon zest in a pan. Simmer for 2-3 minutes and strain, reserving the oil."},{"number": 4, "description": "Process the pancetta/bacon and garlic/rosemary mixture in a food pro

In [28]:
recipe_doc = json.loads(test)
recipe_doc

{'name': 'Pork Roast with Rosemary and Garlic',
 'serving_size': 6,
 'ingredients': [{'id': 1,
   'name': 'pork roast',
   'quantity': 1,
   'unit': 'unit'},
  {'id': 2, 'name': 'fresh rosemary', 'quantity': 2, 'unit': 'tablespoons'},
  {'id': 3, 'name': 'garlic', 'quantity': 8, 'unit': 'cloves'},
  {'id': 4, 'name': 'olive oil', 'quantity': 0.33, 'unit': 'cup'},
  {'id': 5, 'name': 'lemon', 'quantity': 1, 'unit': 'unit'},
  {'id': 6, 'name': 'pancetta or bacon', 'quantity': 4, 'unit': 'ounces'},
  {'id': 7, 'name': 'parsley', 'quantity': 1, 'unit': 'teaspoon'}],
 'steps': [{'number': 1, 'description': 'Preheat the oven to 275°F (135°C)'},
  {'number': 2,
   'description': 'Butterfly the pork roast by cutting into three parts and pounding to an even levelness'},
  {'number': 3,
   'description': 'Mince the garlic and rosemary and combine with olive oil and lemon zest in a pan. Simmer for 2-3 minutes and strain, reserving the oil.'},
  {'number': 4,
   'description': "Process the pancet

In [29]:
recipe_4 = Recipe(**recipe_doc)

In [30]:
recipe_4

Recipe(name='Pork Roast with Rosemary and Garlic', serving_size=6, ingredients=[Ingredient(id=1, name='pork roast', quantity=1.0, unit='unit'), Ingredient(id=2, name='fresh rosemary', quantity=2.0, unit='tablespoons'), Ingredient(id=3, name='garlic', quantity=8.0, unit='cloves'), Ingredient(id=4, name='olive oil', quantity=0.33, unit='cup'), Ingredient(id=5, name='lemon', quantity=1.0, unit='unit'), Ingredient(id=6, name='pancetta or bacon', quantity=4.0, unit='ounces'), Ingredient(id=7, name='parsley', quantity=1.0, unit='teaspoon')], steps=[Step(number=1, description='Preheat the oven to 275°F (135°C)', preparation_time=None, cooking_time=None, used_ingredients=[]), Step(number=2, description='Butterfly the pork roast by cutting into three parts and pounding to an even levelness', preparation_time=None, cooking_time=None, used_ingredients=[]), Step(number=3, description='Mince the garlic and rosemary and combine with olive oil and lemon zest in a pan. Simmer for 2-3 minutes and strai

In [31]:
output_docs = [output.dict() for output in outputs if isinstance(output, Recipe)]

In [32]:
output_docs

[{'name': 'Loaded Mashed Potatoes',
  'serving_size': 4,
  'ingredients': [{'id': 1,
    'name': 'potatoes',
    'quantity': 1000.0,
    'unit': 'grams'},
   {'id': 2, 'name': 'milk', 'quantity': 200.0, 'unit': 'milliliters'},
   {'id': 3, 'name': 'mayonnaise', 'quantity': 200.0, 'unit': 'milliliters'},
   {'id': 4, 'name': 'cheese', 'quantity': 100.0, 'unit': 'grams'},
   {'id': 5, 'name': 'garlic powder', 'quantity': None, 'unit': None},
   {'id': 6, 'name': 'bacon', 'quantity': 12.0, 'unit': 'strips'},
   {'id': 7, 'name': 'butter', 'quantity': None, 'unit': None},
   {'id': 8, 'name': 'green onions', 'quantity': None, 'unit': None},
   {'id': 9, 'name': 'black pepper', 'quantity': None, 'unit': None},
   {'id': 10, 'name': 'salt', 'quantity': None, 'unit': None}],
  'steps': [{'number': 1,
    'description': 'Peel and cut the potatoes into medium-sized pieces.',
    'preparation_time': None,
    'cooking_time': None,
    'used_ingredients': [1]},
   {'number': 2,
    'description':

In [33]:
with open("results.json", "w") as f:
    json.dump(output_docs, f, indent=4)